# Tema do Projeto: Sistema de recomendação de filmes
## Autor: Luiz Fernando da Silva
### Descrição:

Este projeto tem como objetivo desenvolver um sistema de recomendação de filmes simples baseado nos dados retirados do site [Movielens](https://grouplens.org/datasets/movielens/). Essa base de dados foi desenvolvida durante 7 meses por meio do site da MovieLens (movielens.umn.edu) no período de 19 de setembro, 1997 até 22 de abril de 1998. Após a coleta, os dados foram filtrados e os usuário que possuiam menos de 20 avaliações foram removidos. A partir desses dados foi desenvolvido um sistema de recomendação de filmes baseado em dois algoritimos (KNN e SVD) que serão mostrados abaixo.

In [1]:
import pandas as pd
import os
from surprise import Dataset, KNNBasic, Reader, accuracy, SVD
from surprise.model_selection import cross_validate, PredefinedKFold

# Lendo o conjunto de dados
Nos 3 blocos de código abaixo os dados são lidos de modo a criar um dataset do conjunto de treino e de teste, um set de ids de usuários e um set de ids de items (filmes). O conjunto de dados utilizado é o [ml-100k](https://grouplens.org/datasets/movielens/)

In [2]:
items_stream = open('ml-100k/u.item', 'r')
item_data = items_stream.read().split('\n')
item_data = list(map(lambda item: item.split('|')[:2], item_data))
items_stream.close()

In [3]:
database = pd.read_csv('ml-100k/u1.base.csv')
user_set = set(database.user_id)
item_set = set(database.item_id)
not_watch = {user: item_set.difference(database.query('user_id == %s' %(user)).item_id) for user in user_set}

In [4]:
files_dir = os.path.expanduser('ml-100k/')
reader = Reader('ml-100k')

train_file = files_dir + 'u1.base'
test_file = files_dir + 'u1.test'
folds_files = [(train_file, test_file)]

data = Dataset.load_from_folds(folds_files, reader=reader)
pkf = PredefinedKFold()

# Treinando os algoritimos e calculando o RMSE com o conjunto de treino
Nesta análise utilizamos dois diferente algoritimos de recomendação para visualizar as diferenças e semelhanças entre eles. Os algoritimos utilizados formam o KNN e o SVD.

## KNN

* A ideia principal do KNN é determinar o rótulo de classificação de uma amostra baseado nas amostras vizinhas advindas de um conjunto de treinamento.
* Passos:
 + 1-Escolha um vértice arbitrário como vértice atual.
 + 2-Descubra a aresta de menor peso que seja conectada ao vértice atual e a um vértice não visitado V.
 + 3-Faça o vértice atual ser V.
 + 4-Marque V como visitado.
 + 5-Se todos os vértices no domínio estiverem visitados, encerre o algoritmo.
 + 6-Se não vá para o passo 2.
* Para mais detalhes sobre como funciona o algoritimo pela biblioteca [aqui](http://surprise.readthedocs.io/en/stable/knn_inspired.html#surprise.prediction_algorithms.knns.KNNBasic).

## SVD

O algoritmo SVD foi popularizado por Simon Funk durante o Prêmio Netflix. Ele se baseia na fatoração de matrizes para gerar suas predições. Para mais detalhes sobre como funciona o algoritimo pela biblioteca [aqui](http://surprise.readthedocs.io/en/stable/matrix_factorization.html#matrix-factorization-based-algorithms).

In [5]:
sim_options = {
    'name': 'cosine',
    'user_based': True  # compute  similarities between users
}

algo_knn = KNNBasic(sim_options=sim_options, k=4, min_k=2)
algo_svd = SVD()

for trainset, testset in pkf.split(data):

    # train and test algorithm.
    algo_knn.fit(trainset)
    algo_svd.fit(trainset)
    predictions_knn = algo_knn.test(testset)
    predictions_svd = algo_svd.test(testset)
    print('KNN RMSE: %.4f' % accuracy.rmse(predictions_knn, verbose=False))
    print('SVD RMSE: %.4f' % accuracy.rmse(predictions_svd, verbose=False))
    

Computing the cosine similarity matrix...
Done computing similarity matrix.
KNN RMSE: 1.1118
SVD RMSE: 0.9507


# Métodos utilizados para pegar os top k usuários e filmes

Os métodos abaixo foram criados para retornar os 5 filmes que possuem uma estimação de avaliação mais alta utilizando os algoritmos KNN e SVD. E para retornar os 5 usuários mais relacionados ao que está sendo passado por parâmetro.

In [6]:
def get_top_5_knn(uid):
    top = []
    items = not_watch[int(uid)]
    
    for item in items:
        top.append((item, algo_knn.predict(uid=uid, iid=str(item)).est))
    
    return sorted(top, key=lambda item: item[1], reverse=True)[:5]


def get_top_5_movies_knn(uid):
    top_5 = get_top_5_knn(uid)
    return [item_data[int(item[0])][1] for item in top_5]

In [7]:
def get_top_5_svd(uid):
    top = []
    items = not_watch[int(uid)]
    
    for item in items:
        top.append((item, algo_svd.predict(uid=uid, iid=str(item)).est))
    
    return sorted(top, key=lambda item: item[1], reverse=True)[:5]


def get_top_5_movies_svd(uid):
    top_5 = get_top_5_svd(uid)
    return [item_data[int(item[0])][1] for item in top_5]

In [8]:
def get_top_5_neighbors(uid):
    sim_matriz = algo_knn.sim
    inner_uid = algo_knn.trainset.to_inner_uid(uid)
    neighbords = algo_knn.get_neighbors(iid=inner_uid, k=5)
    neighbords_cos = []
    
    for iid in neighbords:
        neighbord = algo_knn.trainset.to_raw_uid(iid)
        cos = sim_matriz[int(uid)-1, int(neighbord)-1]
        
        neighbords_cos.append("%.4f - %.4f" %(float(neighbord), float(cos)))
        
    return neighbords_cos

# Calculando Resultados
Utilizando os métodos acima foram calculadas as recomendações de filmes para o usuário de id 339 usando o KNN e o SVD. Também foi calculado os 5 usuários mais relacionados ao que está sendo passado.

## Aplicado os algoritmos para o usuário de id 339

|         Recomendações KNN            |      Recomendações SVD       | Usuário mais próximos | Similaridade (por distância de cosseno) |
|:------------------------------------:|:----------------------------:|:---------------------:|:--------------:|
| Dead Man Walking (1995)              | Jack (1996)                  |           4           |        1       |
| I.Q. (1994)                          | Army of Darkness (1993)      |          19           |        1       |
| Empire Strikes Back, The (1980)      | Cinema Paradiso (1988)       |          31           |        1       |
| Bridge on the River Kwai, The (1957) | First Wives Club, The (1996) |          33           |        1       |
| Vertigo (1958)                       | Annie Hall (1977)            |          39           |        1       |

## Resultado da análise de precisão dos algoritimos

| Algoritimo |  RMSE  |
|:----------:|:------:|
|    KNN     | 1.1118 |
|    SVD     | 0.9498 |